In [1]:
import random
import tempfile
import os
import subprocess

In [2]:
# Create a vanilla fuzzer
def fuzzer(max_length: int = 100, char_start: int = 32, char_range: int = 32) -> str:
    """A string of up to `max_length` characters
       in the range [`char_start`, `char_start` + `char_range`)"""
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for i in range(0, string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [3]:
fuzzer()

'+\'!$+75)%5/>?3&45! 0."('

In [19]:
fuzzer(1000, ord('a'), 26)

'ggwnlzfhgeimqjzstbjebozhdyduhioqodvpujowimljsjaktpuijsvyivpozxtgktssmplchzzejzzdzqarimoaabwpixwzykvstizzrmalytnglepkgyhwjrjkeczycmhhbsjbjlsiwdzcfouxiizltbbmkvlkzuilnfjpwnyjnvnxndhntuewdpisgcpypgzxqnwynbolkfcikczeruuyrnoxesraxxijvdacznuoskslgodhlxkodqhboiocmmabenoxmwkiyzkccbokhdwnxkzztxqibscczhwxhozigkzzcgqeuutwmxprmiznmchiaxdazxrpgflhqzfsvpzsfjiojsmhaihvnpssrymtpwgojdcybfkowplojsbcapnticyzlitfnxpgdcxdfilwotvrgryfsilctzonegdgydstkgysllagjwjsihnofzrjjyokpoxubdxgjpukhrpppoejafoyplccuhjcabkjuzdopsrboibzbxsezzufizeumodiipxbocfcucqpcfmvzncmrheughunxspjiblknnvlpsvrwmllvdtktjccmtsldsuvshoxpdcqyhznhcracugolxvgqrjiwa'

In [6]:
# Create temp file
FILE = "input.txt"
with open(FILE, "w") as f:
    f.write("2 + 2\n")


In [22]:
# default stdin -> keyboard
# default stdout, stderr -> print to terminal
program = "bc"
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL, # input given from python
                        stdout=subprocess.PIPE,   # output should be returned to python rather than printed
                        stderr=subprocess.PIPE,   # errors >>
                        universal_newlines=True)  # output is treated as text and not as bytes

In [24]:
result.stdout

'4\n'

In [23]:
result.stderr

''

In [25]:
# Fuzz
trials = 1000
program = "bc"

runs = []

for i in range(trials):
    data = fuzzer()
    with open(FILE, "w") as f:
        f.write(data)
    result = subprocess.run([program, FILE],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
    runs.append((data, result))

In [34]:
num_errors = sum(1 for (data, result) in runs if result.stderr != "")
print("%d/%d runs contain errors"% (num_errors, trials))

915/1000 runs contain errors


In [35]:
# Most inputs yield errors

In [40]:
# Let's have a look at the first error
(first_data, first_result) = runs[0]

print(repr(first_data))
print(first_result.stderr)

'%+(9>;&2+#<0/"$,1#0! !=?6&*!*()3-2>,2&6+"&68!+="/ !!6!540\'5=%/-#2;3$'

Parse error: bad token
    input.txt:1




In [45]:
# All of them are parsing errors - No actual bugs found :(
[result.stderr for (data, result) in runs if
 result.stderr != ""
 and "illegal character" not in result.stderr
 and "Parse error" not in result.stderr
 and "syntax error" not in result.stderr
 and "divide by 0" not in result.stderr]

[]